In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('TL_Level_3a_NonPath_SP1_ZyMo_PSL_1_2_3_4_546.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0    14.767060  10.829177  ...        0.032505        0.032749
1    11.728905   8.138424  ...        0.031626        0.032946
2     8.319776   2.773259  ...        0.031596        0.031098
3    10.255686   3.778411  ...        0.031188        0.031233
4     6.745884   2.698354  ...        0.031939        0.030060
..         ...        ...  ...             ...             ...
541   4.938105   2.222147  ...        0.027010        0.029862
542   4.539483   2.269741  ...        0.031191        0.029338
543   4.590172   4.335163  ...        0.030661        0.031927
544  10.080994   8.113971  ...        0.032843        0.031953
545   3.809671   2.913278  ...        0.030617        0.029120

[546 rows x 150 columns]
0      1
1      1
2      1
3      1
4      1
      ..
541    4
542    4
543    4
544    4
545    4
Name: Class, Length: 546, dtype: category
Categories (4, int64): [1, 2, 3, 4]


In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 4 - 1) = 3 components.
  ChangedBehaviorWarn

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 150)            91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 283,716
Trainable params: 283,716
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
20/20 [==============================] - 33s 173ms/step - loss: 0.6791 - accuracy: 0.4294 - val_loss: 0.6028 - val_accuracy: 0.4304
Epoch 2/1000
20/20 [==============================] - 1s 29ms/step - loss: 0.5404 - accuracy: 0.4568 - val_loss: 0.4680 - val_accuracy: 0.4494
Epoch 3/1000
20/20 [==============================] - 1s 30ms/step - loss: 0.4429 - accuracy: 0.4975 - val_loss: 0.4277 - val_accuracy: 0.5253
Epoch 4/1000
20/20 [==============================] - 1s 29ms/step - loss: 0.4345 - accuracy: 0.4944 - val_loss: 0.4205 - val_accuracy: 0.5443
Epoch 5/1000
20/20 [==============================] - 1s 30ms/step - loss: 0.4293 - accuracy: 0.5488 - val_loss: 0.4002 - val_accuracy: 0.6266
Epoch 6/1000
20/20 [==============================] - 1s 29ms/step - loss: 0.3887 - accuracy: 0.5658 - val_loss: 0.3729 - val_accuracy: 0.6203
Epoch 7/1000
20/20 [==============================] - 1s 28ms/step - loss: 0.3727 - accuracy: 0.6093 - val_loss: 0.3603 - val_accuracy: 0.74

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.919047619047619
test-acc = 0.7025316455696202
train-CFR =               precision    recall  f1-score   support

           0       0.99      0.91      0.95       151
           1       0.92      0.88      0.90       161
           2       0.89      0.98      0.93       165
           3       0.88      0.91      0.89       153

    accuracy                           0.92       630
   macro avg       0.92      0.92      0.92       630
weighted avg       0.92      0.92      0.92       630

test-CFR =               precision    recall  f1-score   support

           0       0.95      0.87      0.91        46
           1       0.68      0.58      0.63        36
           2       0.59      0.75      0.66        32
           3       0.59      0.59      0.59        44

    accuracy                           0.70       158
   macro avg       0.70      0.70      0.70       158
weighted avg       0.71      0.70      0.71       158

